# Notebook for inferring predictions and comparing model performance

In [7]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import log_loss

from model_definitions import FullyConnectedNN
from utils.load_splits import load_splits


model = FullyConnectedNN()
model.load_state_dict(torch.load("/faststorage/project/MutationAnalysis/Nimrod/results/models/fc/3sC/model.pth"))

model.eval()

FullyConnectedNN(
  (linear_relu_seq): Sequential(
    (0): LazyLinear(in_features=0, out_features=128, bias=True)
    (1): ReLU()
    (2): LazyLinear(in_features=0, out_features=64, bias=True)
    (3): ReLU()
    (4): LazyLinear(in_features=0, out_features=4, bias=True)
  )
)

In [8]:
_, _, _, _, X_test, y_test = load_splits("3sC")
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

test_probs = []
test_targets = []

with torch.no_grad():
    for xb, yb in test_loader:
        probs = model.predict_proba(xb)

        test_probs.append(probs.cpu())
        test_targets.append(yb.cpu())

test_probs = torch.cat(test_probs).numpy()
test_targets = torch.cat(test_targets).numpy()

loss = log_loss(test_targets, test_probs)
print(f"Test log loss: {loss}")

Test log loss: 0.6922024670608665
